## **_Transformer Model for Daily EV Charging Dataset Process_**

### **Load and Preprocess Data**

``Load Data``

Load the daily EV charging demand dataset.

``Preprocess Data``
- **Normalize Features:**
  - MinMaxScaler.
- **Split Dataset:**
  - Divide the dataset into training (80%), validation (10%) and testing (10%) sets.

### **Define and Compile Model**

``Define Model``

Utilize the transformer model architecture as specified in the referenced paper.

``Compile Model``
- **Optimizer:** Adam
- **Learning Rate:** 1e-4
- **Loss Function:** Mean Squared Error

### **Train the Model**

``Train the model on the training set using the following parameters:``
- **Input Data:** Daily EV charging demand dataset
- **Output:** Forecasted EV charging loads for the next day
- **Training Data Split:** 80% of the dataset
- **Validation Data Split:** 10% of the dataset
- **Testing Data Split:** 10% of the dataset

``Hyperparameters:``
- **Number of Heads:** 1
- **Hidden Dimension of Feedforward Network (dff):** 64
- **Number of Layers:** 6
- **Dropout Rate:** 0.1
- **Epochs:** 60
- **Batch Size:** 32

### **Evaluate the Model**

``Evaluate Model``

- RMSE
- MAE

In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

daily = pd.read_csv('../Dataset/Boulder_Daily.csv')
daily.drop(columns={'Unnamed: 0'}, inplace=True)
daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1668 entries, 0 to 1667
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Energy__kWh_   1668 non-null   float64
 1   Weekday        1668 non-null   object 
 2   Month          1668 non-null   object 
 3   Minimum T      1668 non-null   int64  
 4   Maximum T      1668 non-null   int64  
 5   Snow           1668 non-null   float64
 6   Precipitation  1668 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 91.3+ KB


    Scale the Dataset with MinMaxScaler / One-Hot Encode and Extract the Entire Scaled Dataset

In [36]:
# Define the columns we need to scale and we need to use for One-Hot Encoding
columns_to_scale = ['Energy__kWh_', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation']
categorical_columns = ['Weekday','Month']

# MinMax scaling for numerical columns and One-hot encoding for categorical columns
scaler = MinMaxScaler()
daily_scaled = daily.copy()
daily_scaled[columns_to_scale] = scaler.fit_transform(daily[columns_to_scale])

# One-hot encoding for categorical columns
onehot_encoder = OneHotEncoder(sparse=False)
categorical_encoded = onehot_encoder.fit_transform(daily[categorical_columns])

# Get the feature names from the encoder
encoded_columns = []
for col, values in zip(categorical_columns, onehot_encoder.categories_):
    encoded_columns.extend([f'{col}_{value}' for value in values])

# Create DataFrame with encoded columns
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoded_columns)

# Concatenate the new encoded columns to the original DataFrame
daily_scaled = pd.concat([daily_scaled, categorical_encoded_df], axis=1)

# Drop the original categorical columns
daily_scaled = daily_scaled.drop(categorical_columns, axis=1)
daily_scaled.columns

Index(['Energy__kWh_', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation',
       'Weekday_Friday', 'Weekday_Monday', 'Weekday_Saturday',
       'Weekday_Sunday', 'Weekday_Thursday', 'Weekday_Tuesday',
       'Weekday_Wednesday', 'Month_April', 'Month_August', 'Month_December',
       'Month_February', 'Month_January', 'Month_July', 'Month_June',
       'Month_March', 'Month_May', 'Month_November', 'Month_October',
       'Month_September'],
      dtype='object')

    Divided the dataset into training, testing, and validation datasets according to 0.70, 0.20, and 0.10, respectively.

In [37]:
# Split the dataset into training, validation, and testing sets
def split_dataset(df, train_ratio, val_ratio):

    total_size = len(df)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_df = df[:train_size]
    val_df = df[train_size:train_size + val_size]
    test_df = df[train_size + val_size:]

    assert len(train_df) + len(val_df) + len(test_df) == total_size, "Dataset not split correctly."

    print(f'Training split ratio:   {round(len(train_df) / len(df), 3)}')
    print(f'Validation split ratio: {round(len(val_df) / len(df), 3)}')
    print(f'Testing split ratio:    {round(len(test_df) / len(df), 3)}')
    print("\nShapes of the datasets:")
    print(train_df.shape, val_df.shape, test_df.shape)

    return train_df, val_df, test_df

In [38]:
train_daily_scaled, val_daily_scaled, test_daily_scaled = split_dataset(daily_scaled, train_ratio=0.7, val_ratio=0.2)

Training split ratio:   0.7
Validation split ratio: 0.2
Testing split ratio:    0.101

Shapes of the datasets:
(1167, 24) (333, 24) (168, 24)


    Create sequences for the Transformer model

In [39]:
# Reshape the data
def create_sequences(data, sequence_length):
    inputs = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequence = data.iloc[i:i + sequence_length].values
        target = data.iloc[i + sequence_length]['Energy__kWh_']  # Predict the next value
        inputs.append(sequence)
        targets.append(target)

    inputs_array = np.array(inputs)
    targets_array = np.array(targets)
    
    print(f'Dataset split into sequences:')
    print(f'Sequences shape: {inputs_array.shape}')
    print(f'Targets shape: {targets_array.shape}\n')

    return np.array(inputs), np.array(targets)

In [40]:
sequence_length = 120
num_features = len(daily_scaled.columns)

# Create the training, validation, and test data sequences
train_data_inputs, train_data_targets = create_sequences(train_daily_scaled, sequence_length)
val_data_inputs, val_data_targets = create_sequences(val_daily_scaled, sequence_length)
test_data_inputs, test_data_targets = create_sequences(test_daily_scaled, sequence_length)

Dataset split into sequences:
Sequences shape: (1047, 120, 24)
Targets shape: (1047,)

Dataset split into sequences:
Sequences shape: (213, 120, 24)
Targets shape: (213,)

Dataset split into sequences:
Sequences shape: (48, 120, 24)
Targets shape: (48,)



In [41]:
# The input Datasets must have this input shape (-1, sequence_length, num_features)
train_data_inputs = train_data_inputs.reshape((-1, sequence_length, num_features))
val_data_inputs = val_data_inputs.reshape((-1, sequence_length, num_features))
test_data_inputs = test_data_inputs.reshape((-1, sequence_length, num_features))

train_data_inputs.shape, val_data_inputs.shape, test_data_inputs.shape

((1047, 120, 24), (213, 120, 24), (48, 120, 24))

    Create the Transformer Models

In [42]:
%run "../Code/Transformer.ipynb"

# Define the hyperparameters of the manual model
input_shape = (sequence_length, num_features)
num_heads = 1
d_ff = 64
num_layers = 6
dropout_rate = 0.1
encoder_mask = None
decoder_mask = tf.linalg.band_part(tf.ones((sequence_length, sequence_length)), -1, 0)  # Create a lower triangular mask
decoder_mask = 1 - decoder_mask  # Invert the mask

# Create the transformer model
manul_model = TransformerModel(input_shape, num_heads, d_ff, num_layers, dropout_rate, encoder_mask, decoder_mask)

manul_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 120, 24)]    0           []                               
                                                                                                  
 multi_head_attention_77 (Multi  (None, None, 24)    2400        ['input_5[0][0]',                
 HeadAttention)                                                   'input_5[0][0]',                
                                                                  'input_5[0][0]']                
                                                                                                  
 dropout_159 (Dropout)          (None, None, 24)     0           ['multi_head_attention_77[0][0]']
                                                                                            

In [43]:
# Create the transformer model
keras_model = keras_transformer_model(input_shape, num_heads, d_ff, num_layers, dropout_rate)

keras_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 120, 24)]    0           []                               
                                                                                                  
 multi_head_attention_90 (Multi  (None, 120, 24)     11904       ['input_6[0][0]',                
 HeadAttention)                                                   'input_6[0][0]']                
                                                                                                  
 dropout_186 (Dropout)          (None, 120, 24)      0           ['multi_head_attention_90[0][0]']
                                                                                                  
 layer_normalization_150 (Layer  (None, 120, 24)     48          ['dropout_186[0][0]']      

    Compile the Models

In [44]:
def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(
        tf.keras.backend.mean(
            tf.keras.backend.square(
                y_pred - y_true
            )
        ) + 1e-9
    )

In [45]:
# Define the learning rate for Adam optimizer
learning_rate = 0.01

# Compile the manual model
manul_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse',  metrics=['mae', 'mse', root_mean_squared_error])

# Compile the keras model
keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse',  metrics=['mae', 'mse', root_mean_squared_error])

    Train the Models

In [46]:
# Define the parameters for training
epochs = 50
batch_size = 32

# Convert the data to float32
train_data_inputs = train_data_inputs.astype('float32')
train_data_targets = train_data_targets.astype('float32')

val_data_inputs = val_data_inputs.astype('float32')
val_data_targets = val_data_targets.astype('float32')

train_data_inputs.shape, train_data_targets.shape, val_data_inputs.shape, val_data_targets.shape

((1047, 120, 24), (1047,), (213, 120, 24), (213,))

In [47]:
# Train the manual model
manul_model.fit(train_data_inputs, train_data_targets,
          validation_data=(val_data_inputs, val_data_targets),
          epochs=epochs, batch_size=batch_size)

Epoch 1/50


33/33 [==============================] - 7s 78ms/step - loss: 1.0495 - mae: 0.6552 - mse: 1.0495 - root_mean_squared_error: 0.7369 - val_loss: 0.0281 - val_mae: 0.1384 - val_mse: 0.0281 - val_root_mean_squared_error: 0.1684
Epoch 2/50
33/33 [==============================] - 2s 53ms/step - loss: 0.0710 - mae: 0.2091 - mse: 0.0710 - root_mean_squared_error: 0.2639 - val_loss: 0.0321 - val_mae: 0.1506 - val_mse: 0.0321 - val_root_mean_squared_error: 0.1801
Epoch 3/50
33/33 [==============================] - 2s 55ms/step - loss: 0.0369 - mae: 0.1518 - mse: 0.0369 - root_mean_squared_error: 0.1914 - val_loss: 0.0526 - val_mae: 0.2043 - val_mse: 0.0526 - val_root_mean_squared_error: 0.2307
Epoch 4/50
33/33 [==============================] - 2s 51ms/step - loss: 0.0252 - mae: 0.1265 - mse: 0.0252 - root_mean_squared_error: 0.1577 - val_loss: 0.0561 - val_mae: 0.2123 - val_mse: 0.0561 - val_root_mean_squared_error: 0.2382
Epoch 5/50
33/33 [==============================] - 2s 52ms/step - loss

In [48]:
# Train the keras model
keras_model.fit(train_data_inputs, train_data_targets,
          validation_data=(val_data_inputs, val_data_targets),
          epochs=epochs, batch_size=batch_size)

Epoch 1/50
33/33 [==============================] - 82s 2s/step - loss: 2.5582 - mae: 0.5918 - mse: 2.5582 - root_mean_squared_error: 0.6566 - val_loss: 0.0742 - val_mae: 0.2500 - val_mse: 0.0742 - val_root_mean_squared_error: 0.2736
Epoch 2/50
33/33 [==============================] - 53s 2s/step - loss: 0.0596 - mae: 0.1939 - mse: 0.0596 - root_mean_squared_error: 0.2437 - val_loss: 0.0538 - val_mae: 0.2071 - val_mse: 0.0538 - val_root_mean_squared_error: 0.2331
Epoch 3/50
33/33 [==============================] - 54s 2s/step - loss: 0.0432 - mae: 0.1655 - mse: 0.0432 - root_mean_squared_error: 0.2073 - val_loss: 0.0610 - val_mae: 0.2233 - val_mse: 0.0610 - val_root_mean_squared_error: 0.2482
Epoch 4/50
33/33 [==============================] - 53s 2s/step - loss: 0.0325 - mae: 0.1437 - mse: 0.0325 - root_mean_squared_error: 0.1796 - val_loss: 0.0379 - val_mae: 0.1673 - val_mse: 0.0379 - val_root_mean_squared_error: 0.1956
Epoch 5/50
33/33 [==============================] - 54s 2s/step 

    Evaluate the Models


In [49]:
# Evaluate on validation set
val_metrics_manul = manul_model.evaluate(val_data_inputs, val_data_targets, return_dict=True)
val_metrics_keras = keras_model.evaluate(val_data_inputs, val_data_targets, return_dict=True)

# Evaluate on test set
test_metrics_manul = manul_model.evaluate(test_data_inputs, test_data_targets, return_dict=True)
test_metrics_keras = keras_model.evaluate(test_data_inputs, test_data_targets, return_dict=True)

# Extract individual metrics
val_loss_manul, val_mae_manul, val_mse_manul, val_rmse_manul = val_metrics_manul['loss'], val_metrics_manul['mae'], val_metrics_manul['mse'], val_metrics_manul['root_mean_squared_error']
test_loss_manul, test_mae_manul, test_mse_manul, test_rmse_manul = test_metrics_manul['loss'], test_metrics_manul['mae'], test_metrics_manul['mse'], test_metrics_manul['root_mean_squared_error']

val_loss_keras, val_mae_keras, val_mse_keras, val_rmse_keras = val_metrics_keras['loss'], val_metrics_keras['mae'], val_metrics_keras['mse'], val_metrics_keras['root_mean_squared_error']
test_loss_keras, test_mae_keras, test_mse_keras, test_rmse_keras = test_metrics_keras['loss'], test_metrics_keras['mae'], test_metrics_keras['mse'], test_metrics_keras['root_mean_squared_error']

print('\n\nManual Transformer:\n-------------------')
print(f'Validation Loss: {val_loss_manul}, Validation MSE: {val_mse_manul}, Validation MAE: {val_mae_manul}, Validation RMSE: {val_rmse_manul}')
print(f'Test Loss: {test_loss_manul}, Test MSE: {test_mse_manul}, Test MAE: {test_mae_manul}, Test RMSE: {test_rmse_manul}')

print('\nKeras Transformer:\n------------------')
print(f'Validation Loss: {val_loss_keras}, Validation MSE: {val_mse_keras}, Validation MAE: {val_mae_keras}, Validation RMSE: {val_rmse_keras}')
print(f'Test Loss: {test_loss_keras}, Test MSE: {test_mse_keras}, Test MAE: {test_mae_keras}, Test RMSE: {test_rmse_keras}')

2/2 [==============================] - 1s 192ms/step - loss: 0.2719 - mae: 0.5018 - mse: 0.2719 - root_mean_squared_error: 0.5249


Manual Transformer:
-------------------
Validation Loss: 0.05943479761481285, Validation MSE: 0.05943480134010315, Validation MAE: 0.2198190987110138, Validation RMSE: 0.24495282769203186
Test Loss: 0.24539761245250702, Test MSE: 0.24539761245250702, Test MAE: 0.4746626019477844, Test RMSE: 0.4989876449108124

Keras Transformer:
------------------
Validation Loss: 0.07178176939487457, Validation MSE: 0.07178176939487457, Validation MAE: 0.24538035690784454, Validation RMSE: 0.26915842294692993
Test Loss: 0.27185699343681335, Test MSE: 0.27185699343681335, Test MAE: 0.5017684698104858, Test RMSE: 0.5249119997024536


In [50]:
# Assuming manul_model.predict returns the predictions
val_predictions_manul = manul_model.predict(val_data_inputs)
test_predictions_manul = manul_model.predict(test_data_inputs)

# Assuming keras_model.predict returns the predictions
val_predictions_keras = keras_model.predict(val_data_inputs)
test_predictions_keras  = keras_model.predict(test_data_inputs)

# Calculate MAE and RMSE for validation set
val_mae_manul = np.mean(np.abs(val_data_targets - val_predictions_manul))
val_rmse_manul = np.sqrt(np.mean(np.square(val_data_targets - val_predictions_manul)))

val_mae_keras  = np.mean(np.abs(val_data_targets - val_predictions_keras ))
val_rmse_keras  = np.sqrt(np.mean(np.square(val_data_targets - val_predictions_keras )))

# Calculate MAE and RMSE for test set
test_mae_manul = np.mean(np.abs(test_data_targets - test_predictions_manul))
test_rmse_manul = np.sqrt(np.mean(np.square(test_data_targets - test_predictions_manul)))

test_mae_keras  = np.mean(np.abs(test_data_targets - test_predictions_keras ))
test_rmse_keras  = np.sqrt(np.mean(np.square(test_data_targets - test_predictions_keras )))


print('\n\nManual Transformer:\n-------------------')
print(f'Validation MAE: {val_mae_manul}')
print(f'Validation RMSE: {val_rmse_manul}')
print(f'\nTest MAE: {test_mae_manul}')
print(f'Test RMSE: {test_rmse_manul}')
print('\n==============================')
print('\nKeras Transformer:\n------------------')
print(f'Validation MAE: {val_mae_keras }')
print(f'Validation RMSE: {val_rmse_keras }')
print(f'\nTest MAE: {test_mae_keras }')
print(f'Test RMSE: {test_rmse_keras }')

2/2 [==============================] - 1s 235ms/step


Manual Transformer:
-------------------
Validation MAE: 0.21982847154140472
Validation RMSE: 0.24380365014076233

Test MAE: 0.47466260563791707
Test RMSE: 0.49536998971944574


Keras Transformer:
------------------
Validation MAE: 0.245380237698555
Validation RMSE: 0.2679213285446167

Test MAE: 0.5017678853445425
Test RMSE: 0.5213986479250746


Manual Transformer:
-------------------
Validation MAE: 0.1764928102493286
Validation RMSE: 0.20637793838977814

Test MAE: 0.3902653265425372
Test RMSE: 0.4151124082036069

==============================

Keras Transformer:
------------------
Validation MAE: 0.20550836622714996
Validation RMSE: 0.23427383601665497

Test MAE: 0.42464347135930863
Test RMSE: 0.4475859202184749